In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

#Loading labeled dataset and splitting 80/20 train/test

training = pd.read_csv('labeled_2ch_pikabu.csv')
train_data = pd.DataFrame(training.comment)
train_labels = pd.DataFrame(training.toxic)

first_training_data, first_test_data, first_training_labels, first_test_labels = train_test_split(train_data, train_labels.values, test_size = 0.2)


In [16]:
first_test_data

,comment
11288,Просто с одной из продавщиц замутил док Эмметт...
6794,"Господин Киселев, остановитесь\n"
4086,"И тут я подумал, а что такого со мной случалос..."
14049,а такой электронный анонимный чек? Как посмотр...
1774,Ну и что такого? ко мне тоже 2 мента оружием л...
...,...
11967,Везде поднимают пенсионный возраст. Посмотри н...
8547,даже не стала читать вашу бредовую портняку.\n
146,"Он уже не стесняясь по кд срет, модератор его ..."
13611,"Очевидно же. Вата массово затыкает анусы, чтоб..."


In [17]:
from catboost import CatBoostClassifier, Pool
from catboost.text_processing import Tokenizer

train_pool = Pool(
    first_training_data, 
    first_training_labels,
    text_features=['comment']
)
valid_pool = Pool(
    first_test_data, 
    first_test_labels,
    text_features=['comment']
)

catboost_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'eval_metric': 'Accuracy',
    'task_type': 'CPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 500
}

model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.7851505	test: 0.8380160	best: 0.8380160 (0)	total: 327ms	remaining: 16m 20s
500:	learn: 0.8048400	test: 0.8342005	best: 0.8397503 (2)	total: 1m 24s	remaining: 6m 59s
1000:	learn: 0.8113453	test: 0.8355879	best: 0.8397503 (2)	total: 2m 50s	remaining: 5m 40s
1500:	learn: 0.8253968	test: 0.8487686	best: 0.8498092 (1446)	total: 4m 15s	remaining: 4m 15s
2000:	learn: 0.8319022	test: 0.8498092	best: 0.8511967 (1765)	total: 5m 39s	remaining: 2m 49s
2500:	learn: 0.8388412	test: 0.8498092	best: 0.8511967 (1765)	total: 7m 7s	remaining: 1m 25s
2999:	learn: 0.8454333	test: 0.8508498	best: 0.8511967 (1765)	total: 8m 32s	remaining: 0us

bestTest = 0.8511966701
bestIteration = 1765

Shrink model to first 1766 iterations.


In [22]:
predicts = pd.DataFrame(first_test_data)
predicts['label']=first_test_labels
predicts['predict']= model.predict(first_test_data)
predicts[500:600]

,comment,label,predict
12966,"Сжигатели вон минусут. Ой, и правда, это же би...",1.0,1.0
1679,автор не понимает смысла слова адекватность ор...,1.0,1.0
225,"Ну там то сказано, что нужно отступить 500 мм ...",0.0,0.0
6857,"Зоошиза эгоистичная подъехала, где гарантии кр...",1.0,0.0
6119,"УУУУУ, сюка. Пенис дурачек всем постам дизлайк...",1.0,1.0
...,...,...,...
4888,"не, ну зеленый банк нужен из-за того, что он е...",0.0,0.0
6464,НАВАЛЬНЫЙ ХУЙ ПАТЛАТЫЙ. ОН МНЕ ОМЕРЗИТЕЛЕН ТЕМ...,1.0,1.0
6354,"Че ты тут пишешь, гейройская пидорашка? Скорее...",1.0,1.0
13543,"Готов подписаться - просвети. Я считал, что лю...",0.0,0.0


In [94]:
data_test=pd.read_excel('first_try_lentach_predictions.xlsx')
data_test=data_test.iloc[:,1]
data_test=pd.DataFrame(data_test)
#data_test['predict']=model.predict(data_test)
data_test['predict']=['' for x in range(len(data_test))]

In [93]:
data_test['predict'][1000:1050]=model.predict(data_test.values.tolist()[1000:1050])
data_test[1000:1050]

,0,predict
1000,"Интересно, а Марк знает, что уже более 20 лет ...",0.0
1001,"Захотели - подняли, передумали - опустили...\n...",1.0
1002,"по крайней мере, инвесторы вкинули бабки в реа...",0.0
1003,Офигенные инвесторы актив перепутали)) Хомячье...,0.0
1004,Надо было купить их акции и сегодня продать (((,0.0
1005,ингавирин же вроде пустышка не?,0.0
1006,"""...исследуют Ингавирин в качестве лекарства п...",0.0
1007,И еще одна новость в тему. Производитель Ингав...,1.0
1008,"Вопрос: кто производит этот Ингавирин, нет ли ...",0.0
1009,ингавирин? сириусли?)) ученые исследуют?))) а ...,0.0
